# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [11]:
# imports

import os
import json
from dotenv import load_dotenv
from groq import Groq
from agents.deals import ScrapedDeal, DealSelection

In [2]:
# Initialize and constants

load_dotenv(override=True)
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
MODEL = "deepseek-r1-distill-llama-70b"

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [04:16<00:00, 51.36s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

'Title: Giselle 33" Bar Stool for $99 + free shipping\nDetails: That\'s a savings of $300 on a single. Note that a set of two is $169, or $84.50 each. Buy Now at hernest.com\nFeatures: 300 lbs. weight capacity Carbon steel frame PU Leather upholstery\nURL: https://www.dealnews.com/Giselle-33-Bar-Stool-for-99-free-shipping/21733669.html?iref=rss-c196'

In [ ]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [16]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals[:40]])

In [17]:
client = Groq()
def get_recommendations():
    """Fetch deal recommendations from the Groq model and parse into DealSelection"""
    completion = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={"type": "json_object"}
    )
    
    # Parse the JSON response into the DealSelection pydantic model
    content = completion.choices[0].message.content
    data = json.loads(content)
    result = DealSelection(**data)
    return result


In [18]:
result = get_recommendations()

In [20]:
len(result.deals)

5

In [21]:
result.deals[1]

Deal(product_description='Kospet Tank T2 Smartwatch comes with a 1.43" AMOLED display, offering a battery life of 50+ days standby and 10-15 days of daily usage. It has an IP69K water rating and supports up to 70 sports modes, including hiking and skiing. Additional features include heart and blood pressure monitoring.', price=59.99, url='https://www.dealnews.com/Kospet-Tank-T2-Smartwatch-Clearance-Sale-for-60-free-shipping/21733599.html?iref=rss-c142')

In [26]:
from agents.scanner_agent import ScannerAgent

In [ ]:
agent = ScannerAgent()
result = agent.scan()

In [ ]:
result